# Walmart Sales Data Analysis

## Overview
This notebook provides a comprehensive analysis of Walmart sales data to identify trends, seasonality, and product performance patterns.

### Dataset Information
- **Source**: Walmart Recruiting - Store Sales Forecasting (Kaggle)
- **Period**: February 2010 to November 2012
- **Stores**: 45 Walmart stores across different regions
- **Departments**: 99 departments per store


## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime, timedelta
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

In [ ]:
# Connect to SQLite database
db_path = '../data/walmart_sales.db'
conn = sqlite3.connect(db_path)

print(f"Connected to database: {db_path}")

# Load datasets
train = pd.read_sql_query("SELECT * FROM train", conn)
stores = pd.read_sql_query("SELECT * FROM stores", conn)
features = pd.read_sql_query("SELECT * FROM features", conn)

# Convert date columns
train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

print(f"Train dataset shape: {train.shape}")
print(f"Stores dataset shape: {stores.shape}")
print(f"Features dataset shape: {features.shape}")

## 2. Data Exploration and Quality Check

In [ ]:
# Basic dataset information
print("=== TRAIN DATASET INFO ===")
print(train.info())
print("\n=== TRAIN DATASET DESCRIPTION ===")
print(train.describe())
print("\n=== MISSING VALUES ===")
print(train.isnull().sum())

In [ ]:
# Basic statistics
print("=== KEY STATISTICS ===")
print(f"Total records: {len(train):,}")
print(f"Unique stores: {train['Store'].nunique()}")
print(f"Unique departments: {train['Dept'].nunique()}")
print(f"Date range: {train['Date'].min()} to {train['Date'].max()}")
print(f"Total sales: ${train['Weekly_Sales'].sum():,.2f}")
print(f"Average weekly sales: ${train['Weekly_Sales'].mean():,.2f}")
print(f"Negative sales count: {(train['Weekly_Sales'] < 0).sum()}")

## 3. Sales Trends Analysis

In [ ]:
# Time series analysis
# Monthly sales trends
monthly_sales = train.groupby(train['Date'].dt.to_period('M')).agg({
    'Weekly_Sales': ['sum', 'mean', 'count'],
    'Store': 'nunique',
    'Dept': 'nunique'
}).round(2)

monthly_sales.columns = ['Total_Sales', 'Avg_Sales', 'Record_Count', 'Active_Stores', 'Active_Depts']
monthly_sales.index = monthly_sales.index.to_timestamp()

print("Monthly Sales Summary:")
print(monthly_sales.head(10))

In [ ]:
# Plot monthly sales trends
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Total monthly sales
ax1.plot(monthly_sales.index, monthly_sales['Total_Sales'], marker='o', linewidth=2)
ax1.set_title('Total Monthly Sales Trend', fontsize=14, fontweight='bold')
ax1.set_ylabel('Total Sales ($)')
ax1.grid(True, alpha=0.3)

# Average weekly sales
ax2.plot(monthly_sales.index, monthly_sales['Avg_Sales'], marker='s', color='orange', linewidth=2)
ax2.set_title('Average Weekly Sales Trend', fontsize=14, fontweight='bold')
ax2.set_ylabel('Average Sales ($)')
ax2.grid(True, alpha=0.3)

# Yearly comparison
yearly_sales = train.groupby(train['Date'].dt.year)['Weekly_Sales'].sum()
ax3.bar(yearly_sales.index, yearly_sales.values, color='skyblue', alpha=0.8)
ax3.set_title('Total Sales by Year', fontsize=14, fontweight='bold')
ax3.set_ylabel('Total Sales ($)')
ax3.set_xlabel('Year')

# Quarterly trends
quarterly_sales = train.groupby([train['Date'].dt.year, train['Date'].dt.quarter])['Weekly_Sales'].sum()
quarters = [f"{year}-Q{quarter}" for year, quarter in quarterly_sales.index]
ax4.plot(range(len(quarterly_sales)), quarterly_sales.values, marker='d', color='green', linewidth=2)
ax4.set_title('Quarterly Sales Trend', fontsize=14, fontweight='bold')
ax4.set_ylabel('Total Sales ($)')
ax4.set_xlabel('Quarter')
ax4.set_xticks(range(0, len(quarters), 2))
ax4.set_xticklabels([quarters[i] for i in range(0, len(quarters), 2)], rotation=45)

plt.tight_layout()
plt.show()

## 4. Store Performance Analysis

In [ ]:
# Merge train data with store information
train_stores = train.merge(stores, on='Store')

# Top performing stores
top_stores = train_stores.groupby(['Store', 'Type', 'Size']).agg({
    'Weekly_Sales': ['sum', 'mean', 'count'],
    'Dept': 'nunique'
}).round(2)

top_stores.columns = ['Total_Sales', 'Avg_Sales', 'Record_Count', 'Dept_Count']
top_stores = top_stores.sort_values('Total_Sales', ascending=False)

print("Top 15 Performing Stores:")
print(top_stores.head(15))

In [ ]:
# Store performance by type
store_type_performance = train_stores.groupby('Type').agg({
    'Store': 'nunique',
    'Weekly_Sales': ['sum', 'mean'],
    'Size': 'mean'
}).round(2)

store_type_performance.columns = ['Store_Count', 'Total_Sales', 'Avg_Sales', 'Avg_Size']

print("Store Performance by Type:")
print(store_type_performance)

# Visualize store performance by type
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Total sales by store type
ax1.bar(store_type_performance.index, store_type_performance['Total_Sales'], 
        color=['skyblue', 'lightcoral', 'lightgreen'][:len(store_type_performance)])
ax1.set_title('Total Sales by Store Type', fontsize=14, fontweight='bold')
ax1.set_ylabel('Total Sales ($)')
ax1.set_xlabel('Store Type')

# Average sales by store type
ax2.bar(store_type_performance.index, store_type_performance['Avg_Sales'], 
        color=['skyblue', 'lightcoral', 'lightgreen'][:len(store_type_performance)])
ax2.set_title('Average Sales by Store Type', fontsize=14, fontweight='bold')
ax2.set_ylabel('Average Sales ($)')
ax2.set_xlabel('Store Type')

plt.tight_layout()
plt.show()

## 5. Department Performance Analysis

In [ ]:
# Department performance analysis
dept_performance = train.groupby('Dept').agg({
    'Weekly_Sales': ['sum', 'mean', 'std', 'count'],
    'Store': 'nunique'
}).round(2)

dept_performance.columns = ['Total_Sales', 'Avg_Sales', 'Sales_Std', 'Record_Count', 'Store_Count']
dept_performance['CV'] = (dept_performance['Sales_Std'] / dept_performance['Avg_Sales'] * 100).round(2)
dept_performance = dept_performance.sort_values('Total_Sales', ascending=False)

print("Top 20 Performing Departments:")
print(dept_performance.head(20))

In [ ]:
# Visualize top departments
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Top 15 departments by total sales
top_15_depts = dept_performance.head(15)
ax1.barh(range(len(top_15_depts)), top_15_depts['Total_Sales'], color='steelblue')
ax1.set_yticks(range(len(top_15_depts)))
ax1.set_yticklabels([f"Dept {idx}" for idx in top_15_depts.index])
ax1.set_title('Top 15 Departments by Total Sales', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Sales ($)')

# Department sales distribution
ax2.hist(dept_performance['Total_Sales'], bins=20, color='lightcoral', alpha=0.7)
ax2.set_title('Distribution of Department Total Sales', fontsize=14, fontweight='bold')
ax2.set_xlabel('Total Sales ($)')
ax2.set_ylabel('Number of Departments')

# Most consistent departments (lowest CV)
most_consistent = dept_performance[dept_performance['Record_Count'] >= 100].sort_values('CV').head(15)
ax3.barh(range(len(most_consistent)), most_consistent['CV'], color='lightgreen')
ax3.set_yticks(range(len(most_consistent)))
ax3.set_yticklabels([f"Dept {idx}" for idx in most_consistent.index])
ax3.set_title('Most Consistent Departments (Lowest Variability)', fontsize=14, fontweight='bold')
ax3.set_xlabel('Coefficient of Variation (%)')

# Average sales by department (top 15)
top_15_avg = dept_performance.head(15)
ax4.bar(range(len(top_15_avg)), top_15_avg['Avg_Sales'], color='gold')
ax4.set_xticks(range(len(top_15_avg)))
ax4.set_xticklabels([f"D{idx}" for idx in top_15_avg.index], rotation=45)
ax4.set_title('Average Sales - Top 15 Departments', fontsize=14, fontweight='bold')
ax4.set_ylabel('Average Sales ($)')

plt.tight_layout()
plt.show()

## 6. Seasonality Analysis

In [ ]:
# Seasonality patterns
# Monthly patterns
monthly_pattern = train.groupby(train['Date'].dt.month).agg({
    'Weekly_Sales': ['sum', 'mean', 'count']
}).round(2)

monthly_pattern.columns = ['Total_Sales', 'Avg_Sales', 'Record_Count']
monthly_pattern.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

print("Monthly Sales Patterns:")
print(monthly_pattern)

# Day of week patterns
train['DayOfWeek'] = train['Date'].dt.day_name()
dow_pattern = train.groupby('DayOfWeek')['Weekly_Sales'].agg(['sum', 'mean', 'count']).round(2)
dow_pattern.columns = ['Total_Sales', 'Avg_Sales', 'Record_Count']

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_pattern = dow_pattern.reindex(day_order)

print("\nDay of Week Patterns:")
print(dow_pattern)

In [ ]:
# Visualize seasonality
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Monthly sales pattern
ax1.plot(monthly_pattern.index, monthly_pattern['Avg_Sales'], marker='o', linewidth=3, markersize=8)
ax1.set_title('Average Sales by Month', fontsize=14, fontweight='bold')
ax1.set_ylabel('Average Sales ($)')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Day of week pattern
ax2.bar(dow_pattern.index, dow_pattern['Avg_Sales'], color='lightcoral')
ax2.set_title('Average Sales by Day of Week', fontsize=14, fontweight='bold')
ax2.set_ylabel('Average Sales ($)')
ax2.tick_params(axis='x', rotation=45)

# Holiday impact
holiday_impact = train.groupby('IsHoliday')['Weekly_Sales'].agg(['mean', 'count']).round(2)
holiday_impact.columns = ['Avg_Sales', 'Record_Count']
holiday_impact.index = ['Non-Holiday', 'Holiday']

ax3.bar(holiday_impact.index, holiday_impact['Avg_Sales'], color=['lightblue', 'orange'])
ax3.set_title('Sales Impact: Holiday vs Non-Holiday', fontsize=14, fontweight='bold')
ax3.set_ylabel('Average Sales ($)')

# Quarterly trend
quarterly_trend = train.groupby([train['Date'].dt.year, train['Date'].dt.quarter])['Weekly_Sales'].mean()
quarters_labels = [f"{year}-Q{quarter}" for year, quarter in quarterly_trend.index]

ax4.plot(range(len(quarterly_trend)), quarterly_trend.values, marker='s', linewidth=2, markersize=6)
ax4.set_title('Average Sales by Quarter', fontsize=14, fontweight='bold')
ax4.set_ylabel('Average Sales ($)')
ax4.set_xlabel('Quarter')
ax4.set_xticks(range(0, len(quarters_labels), 2))
ax4.set_xticklabels([quarters_labels[i] for i in range(0, len(quarters_labels), 2)], rotation=45)

plt.tight_layout()
plt.show()

print(f"\nHoliday Impact:")
print(holiday_impact)

## 7. Advanced Analytics

In [ ]:
# Moving averages and trends
weekly_sales_total = train.groupby('Date')['Weekly_Sales'].sum().reset_index()
weekly_sales_total = weekly_sales_total.sort_values('Date')

# Calculate moving averages
weekly_sales_total['MA_4_weeks'] = weekly_sales_total['Weekly_Sales'].rolling(window=4).mean()
weekly_sales_total['MA_8_weeks'] = weekly_sales_total['Weekly_Sales'].rolling(window=8).mean()
weekly_sales_total['MA_12_weeks'] = weekly_sales_total['Weekly_Sales'].rolling(window=12).mean()

# Plot moving averages
plt.figure(figsize=(15, 8))
plt.plot(weekly_sales_total['Date'], weekly_sales_total['Weekly_Sales'], 
         alpha=0.5, label='Weekly Sales', linewidth=1)
plt.plot(weekly_sales_total['Date'], weekly_sales_total['MA_4_weeks'], 
         label='4-Week MA', linewidth=2)
plt.plot(weekly_sales_total['Date'], weekly_sales_total['MA_8_weeks'], 
         label='8-Week MA', linewidth=2)
plt.plot(weekly_sales_total['Date'], weekly_sales_total['MA_12_weeks'], 
         label='12-Week MA', linewidth=2)

plt.title('Sales Trends with Moving Averages', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Total Weekly Sales ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
# Merge with features data
train_features = train.merge(features, on=['Store', 'Date'], how='left')

# Select numeric columns for correlation
numeric_cols = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
correlation_data = train_features[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix: Sales vs External Factors', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("Correlation with Weekly Sales:")
sales_correlations = correlation_data['Weekly_Sales'].sort_values(ascending=False)
for factor, corr in sales_correlations.items():
    if factor != 'Weekly_Sales':
        print(f"{factor:.<20} {corr:.3f}")

## 8. Key Insights and Summary

In [ ]:
# Generate summary insights
print("=" * 60)
print("WALMART SALES ANALYSIS - KEY INSIGHTS")
print("=" * 60)

# 1. Overall Performance
total_sales = train['Weekly_Sales'].sum()
avg_weekly_sales = train['Weekly_Sales'].mean()
print(f"\n1. OVERALL PERFORMANCE")
print(f"   Total Sales: ${total_sales:,.2f}")
print(f"   Average Weekly Sales: ${avg_weekly_sales:,.2f}")
print(f"   Analysis Period: {train['Date'].min()} to {train['Date'].max()}")

# 2. Top Performers
best_store = top_stores.index[0][0]
best_dept = dept_performance.index[0]
print(f"\n2. TOP PERFORMERS")
print(f"   Best Store: Store {best_store} (${top_stores.iloc[0]['Total_Sales']:,.2f})")
print(f"   Best Department: Dept {best_dept} (${dept_performance.iloc[0]['Total_Sales']:,.2f})")
print(f"   Best Store Type: {store_type_performance.index[0]}")

# 3. Seasonal Patterns
best_month = monthly_pattern.idxmax()['Avg_Sales']
worst_month = monthly_pattern.idxmin()['Avg_Sales']
print(f"\n3. SEASONAL PATTERNS")
print(f"   Peak Sales Month: {best_month}")
print(f"   Lowest Sales Month: {worst_month}")
print(f"   Holiday Impact: {holiday_impact.loc['Holiday', 'Avg_Sales'] - holiday_impact.loc['Non-Holiday', 'Avg_Sales']:+.2f}")

# 4. Store Analysis
print(f"\n4. STORE ANALYSIS")
print(f"   Total Stores: {train['Store'].nunique()}")
print(f"   Store Types: {stores['Type'].nunique()} ({', '.join(stores['Type'].unique())})")
print(f"   Average Store Size: {stores['Size'].mean():,.0f} sq ft")

# 5. Department Analysis
consistent_dept = dept_performance[dept_performance['Record_Count'] >= 100].sort_values('CV').index[0]
volatile_dept = dept_performance[dept_performance['Record_Count'] >= 100].sort_values('CV', ascending=False).index[0]
print(f"\n5. DEPARTMENT ANALYSIS")
print(f"   Total Departments: {train['Dept'].nunique()}")
print(f"   Most Consistent Dept: {consistent_dept} (CV: {dept_performance.loc[consistent_dept, 'CV']:.1f}%)")
print(f"   Most Volatile Dept: {volatile_dept} (CV: {dept_performance.loc[volatile_dept, 'CV']:.1f}%)")

print("\n" + "=" * 60)
print("Analysis completed successfully!")
print("=" * 60)

## 9. Export Results

In [ ]:
# Export key results to CSV files
import os

# Create exports directory
exports_dir = '../reports'
os.makedirs(exports_dir, exist_ok=True)

# Export summaries
top_stores.to_csv(f'{exports_dir}/top_stores_performance.csv')
dept_performance.to_csv(f'{exports_dir}/department_performance.csv')
monthly_pattern.to_csv(f'{exports_dir}/monthly_sales_patterns.csv')
store_type_performance.to_csv(f'{exports_dir}/store_type_performance.csv')

print("Results exported to CSV files:")
print(f"- {exports_dir}/top_stores_performance.csv")
print(f"- {exports_dir}/department_performance.csv")
print(f"- {exports_dir}/monthly_sales_patterns.csv")
print(f"- {exports_dir}/store_type_performance.csv")

# Close database connection
conn.close()
print("\nDatabase connection closed.")